In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.168, 0.083, 0.107, 0.111],
    [0.090, 0.078, 0.103, 0.102],
    [0.060, 0.075, 0.083, 0.088],
    [0.073, 0.091, 0.242, 0.078],
    [0.249, 0.063, 0.120, 0.068],
    [0.033, 0.139, 0.020, 0.144],
    [0.025, 0.095, 0.025, 0.148],
    [0.032, 0.130, 0.116, 0.105],
    [0.091, 0.103, 0.128, 0.089], 
    [0.012, 0.144, 0.044, 0.069]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


           C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  \
Site4   0.073  0.091  0.242  0.078      0.414431       0.058251  0.356180   
Site8   0.032  0.130  0.116  0.105      0.188197       0.064776  0.123420   
Site9   0.091  0.103  0.128  0.089      0.167292       0.073551  0.093741   
Site1   0.168  0.083  0.107  0.111      0.114933       0.114226  0.000706   
Site5   0.249  0.063  0.120  0.068      0.143855       0.151727 -0.007872   
Site2   0.090  0.078  0.103  0.102      0.087214       0.134845 -0.047631   
Site10  0.012  0.144  0.044  0.069      0.136500       0.194730 -0.058230   
Site6   0.033  0.139  0.020  0.144      0.123636       0.234563 -0.110927   
Site3   0.060  0.075  0.083  0.088      0.055676       0.182501 -0.126825   
Site7   0.025  0.095  0.025  0.148      0.038563       0.261127 -0.222563   

        Rank  
Site4      1  
Site8      2  
Site9      3  
Site1      4  
Site5      5  
Site2      6  
Site10     7  
Site6      8  
Site3      9  
Si